In [52]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Para estandarizar características
from tensorflow.keras.models import Sequential  # Para inicializar modelos secuenciales
from tensorflow.keras.layers import Dense, Input  # Para definir capas densamente conectadas en la red neuronal
from sklearn.impute import SimpleImputer  # Para imputar valores faltantes
import numpy as np  # Para cálculos numéricos
from sklearn.metrics import mean_squared_error, r2_score  # Para calcular métricas de rendimiento del modelo

In [53]:
# Cargar datos
df = pd.read_csv("jugadores_fantasy_ultrainstinto.csv")


# Imprimir las columnas existentes y la columna posicion 
print(df.columns)
print("\n")
df

Nombre                   0
Precio                   0
Media                    0
Total_puntos             0
posicion                 0
equipo                   0
Partidos               124
Minutos                124
Goles                  124
Asistencias            124
Asistencias sin Gol    124
Centros                124
Despejes               124
Remates                124
Tiros                  124
Recuperaciones         124
Perdidas               124
Penaltis fallados      124
Goles en contra        124
Rojas                  124
Paradas                124
Penaltis cometidos     124
Amarillas              124
Seguna Amarilla        124
Penaltis provocados    124
Penaltis parados       124
Goles en PP            124
dtype: int64


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'J. Bellingham'

In [33]:
# Dividir los datos en características (X) y el objetivo (y)
X_Todo = df[['Media', 'Partidos', 'Minutos', 'Goles', 'Asistencias', 'Asistencias sin Gol', 'Centros', 'Despejes', 'Remates', 'Tiros', 'Recuperaciones', 'Perdidas', 'Penaltis fallados', 'Goles en contra', 'Rojas', 'Paradas', 'Penaltis cometidos', 'Amarillas', 'Seguna Amarilla', 'Penaltis provocados', 'Penaltis parados', 'Goles en PP']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_puntos, X_test_puntos, y_puntos_train, y_puntos_test = train_test_split(X_Todo, y_puntos, test_size=0.2, random_state=42)
X_train_precio, X_test_precio, y_precio_train, y_precio_test = train_test_split(X_Todo, y_precio, test_size=0.2, random_state=42)


# Escalar las características
scaler = StandardScaler()
X_train_puntos_scaled = scaler.fit_transform(X_train_puntos)
X_test_puntos_scaled = scaler.transform(X_test_puntos)
X_train_precio_scaled = scaler.fit_transform(X_train_precio)
X_test_precio_scaled = scaler.transform(X_test_precio)

In [34]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Input(shape=(X_train_puntos_scaled.shape[1],)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_puntos_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [35]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Input(shape=(X_train_precio_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_precio_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)

In [36]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_puntos_scaled).flatten()
precio_predictions = model_precio.predict(X_test_precio_scaled).flatten()

# Crear un DataFrame con las predicciones y el índice original
predictions_df = pd.DataFrame({
    'Predicted_Points': puntos_predictions,
    'Predicted_Price': precio_predictions
}, index=X_test_puntos.index)

# Unir las predicciones con el DataFrame original usando el índice
df_with_predictions = df.join(predictions_df, how='inner')

# Ordenar el DataFrame de forma descendente por precio predicho y puntos predichos
df_sorted = df_with_predictions.sort_values(by=['Predicted_Price', 'Predicted_Points'], ascending=False)

# Impresión de la lista de jugadores ordenados por mayor precio y puntos predichos
print("Lista de jugadores ordenados por mayor precio y puntos predichos:")
print(df_sorted[['Nombre', 'Predicted_Price', 'Predicted_Points']])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Lista de jugadores ordenados por mayor precio y puntos predichos:
            Nombre  Predicted_Price  Predicted_Points
0    J. Bellingham              NaN               NaN
2     A. Griezmann              NaN               NaN
9       A. Budimir              NaN               NaN
11           Sávio              NaN               NaN
18     F. Valverde              NaN               NaN
..             ...              ...               ...
491       A. López              NaN               NaN
496       I. Grbić              NaN               NaN
497    J. Doménech              NaN               NaN
500       P. Marín              NaN               NaN
501       P. Reina              NaN               NaN

[102 rows x 3 columns]


In [37]:
# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print("Calculo del error cuadratico y coeficiente de determinacion por puntos:")
print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print("\nCalculo del error cuadratico y coeficiente de determinacion por precios")
print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


ValueError: Input contains NaN.

In [38]:
# Crear DataFrames para los top 10 jugadores por precio y puntos predichos
top_10_precio = df_sorted.nlargest(10, 'Predicted_Price')
top_10_puntos = df_sorted.nlargest(10, 'Predicted_Points')

# Filtrar jugadores con la posición de delantero
delanteros = df_with_predictions[df_with_predictions['posicion'] == 'delantero']
mediocentro = df_with_predictions[df_with_predictions['posicion'] == 'mediocentro']
portero = df_with_predictions[df_with_predictions['posicion'] == 'portero']
defensa = df_with_predictions[df_with_predictions['posicion'] == 'defensa']

# Crear DataFrames para los top 10 Delanteros por precio y puntos predichos
top_10_precio_Delanteros = delanteros.nlargest(10, 'Predicted_Price')
top_10_puntos_Delanteros = delanteros.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Mediocentros por precio y puntos predichos
top_10_precio_Mediocentro = mediocentro.nlargest(10, 'Predicted_Price')
top_10_puntos_Mediocentro = mediocentro.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Porteros por precio y puntos predichos
top_10_precio_Portero = portero.nlargest(10, 'Predicted_Price')
top_10_puntos_Portero = portero.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Defensas por precio y puntos predichos
top_10_precio_Defensa = defensa.nlargest(10, 'Predicted_Price')
top_10_puntos_Defensa = defensa.nlargest(10, 'Predicted_Points')


In [39]:
# Guardar los DataFrames en archivos CSV
top_10_precio.to_csv('top_10_precio_predictions.csv', index=False)
top_10_puntos.to_csv('top_10_puntos_predictions.csv', index=False)

top_10_precio_Delanteros.to_csv('top_10_precio_delanteros_predictions.csv', index=False)
top_10_puntos_Delanteros.to_csv('top_10_puntos_delanteros_predictions.csv', index=False)

top_10_precio_Mediocentro.to_csv('top_10_precio_mediocentros_predictions.csv', index=False)
top_10_puntos_Mediocentro.to_csv('top_10_puntos_mediocentros_predictions.csv', index=False)

top_10_precio_Portero.to_csv('top_10_precio_porteros_predictions.csv', index=False)
top_10_puntos_Portero.to_csv('top_10_puntos_porteros_predictions.csv', index=False)

top_10_precio_Defensa.to_csv('top_10_precio_defensas_predictions.csv', index=False)
top_10_puntos_Defensa.to_csv('top_10_puntos_defensa_predictions.csv', index=False)


In [40]:
# Mostrar los DataFrames creados
print("Top 10 jugadores por precio predicho:")
print(top_10_precio[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 jugadores por puntos predichos:")
print(top_10_puntos[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 delanteros por precio predicho:")
print(top_10_precio_Delanteros[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 delanteros por puntos predichos:")
print(top_10_puntos_Delanteros[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 mediocentros por precio predicho:")
print(top_10_precio_Mediocentro[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 mediocentros por puntos predichos:")
print(top_10_puntos_Mediocentro[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 porteros por precio predicho:")
print(top_10_precio_Portero[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 porteros por puntos predichos:")
print(top_10_puntos_Portero[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 defensas por precio predicho:")
print(top_10_precio_Defensa[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 defensas por puntos predichos:")
print(top_10_puntos_Defensa[['Nombre', 'Predicted_Price', 'Predicted_Points']])

Top 10 jugadores por precio predicho:
           Nombre  Predicted_Price  Predicted_Points
0   J. Bellingham              NaN               NaN
2    A. Griezmann              NaN               NaN
9      A. Budimir              NaN               NaN
11          Sávio              NaN               NaN
18    F. Valverde              NaN               NaN
22      G. Moreno              NaN               NaN
30      B. Méndez              NaN               NaN
33     S. Arribas              NaN               NaN
39     I. Palazón              NaN               NaN
46       S. Costa              NaN               NaN
Top 10 jugadores por puntos predichos:
           Nombre  Predicted_Price  Predicted_Points
0   J. Bellingham              NaN               NaN
2    A. Griezmann              NaN               NaN
9      A. Budimir              NaN               NaN
11          Sávio              NaN               NaN
18    F. Valverde              NaN               NaN
22      G. Moreno     

In [41]:
top_10_precio

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points,Predicted_Price
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,13.0,1025.0,11.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,13.0,1071.0,9.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,14.0,966.0,1.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
11,Sávio,18030000,6.5,123,delantero,Girona Fútbol Club,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,F. Valverde,20115000,5.7,108,mediocentro,Real Madrid Club de Fútbol,14.0,1118.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN
22,G. Moreno,18341000,6.5,104,delantero,Villarreal Club de Fútbol,11.0,804.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,10.0,866.0,2.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
33,S. Arribas,11043000,5.1,96,mediocentro,Unión Deportiva Almería,13.0,893.0,3.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
39,I. Palazón,12991000,4.8,92,mediocentro,Rayo Vallecano de Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,S. Costa,9796000,4.7,89,mediocentro,Real Club Deportivo Mallorca,11.0,655.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [42]:
top_10_precio_Delanteros

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points,Predicted_Price
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,13.0,1071.0,9.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,14.0,966.0,1.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
11,Sávio,18030000,6.5,123,delantero,Girona Fútbol Club,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,G. Moreno,18341000,6.5,104,delantero,Villarreal Club de Fútbol,11.0,804.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN
72,V. Muriqi,12454000,6.8,81,delantero,Real Club Deportivo Mallorca,12.0,942.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
73,F. Torres,8080000,4.5,81,delantero,Fútbol Club Barcelona,12.0,948.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
75,M. Uzuni,3324000,4.3,81,delantero,Granada Club de Fútbol,13.0,653.0,1.0,3.0,...,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,NaN,NaN
82,S. Omorodion,5889000,4.4,79,delantero,Deportivo Alavés,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,L. Ramazani,3171000,4.2,79,delantero,Unión Deportiva Almería,13.0,751.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
86,A. Sørloth,9176000,4.8,77,delantero,Villarreal Club de Fútbol,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
top_10_precio_Mediocentro

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points,Predicted_Price
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,13.0,1025.0,11.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
18,F. Valverde,20115000,5.7,108,mediocentro,Real Madrid Club de Fútbol,14.0,1118.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,10.0,866.0,2.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
33,S. Arribas,11043000,5.1,96,mediocentro,Unión Deportiva Almería,13.0,893.0,3.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
39,I. Palazón,12991000,4.8,92,mediocentro,Rayo Vallecano de Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,S. Costa,9796000,4.7,89,mediocentro,Real Club Deportivo Mallorca,11.0,655.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
55,Y. Herrera,11017000,5.7,86,mediocentro,Girona Fútbol Club,11.0,729.0,1.0,5.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,NaN,NaN
68,V. Tsygankov,14504000,5.9,82,mediocentro,Girona Fútbol Club,12.0,887.0,1.0,5.0,...,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,NaN,NaN
69,R. Alcaraz,4127000,4.6,82,mediocentro,Cádiz Club de Fútbol,14.0,899.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
70,M. Llorente,10543000,4.3,82,mediocentro,Club Atlético de Madrid,10.0,584.0,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN


In [44]:
top_10_precio_Portero

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points,Predicted_Price
392,R. Fernández,160000,3.3,13,portero,Granada Club de Fútbol,13.0,373.0,2.0,1.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
436,D. Mariño,189000,3.0,3,portero,Unión Deportiva Almería,14.0,90.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
474,T. Courtois,368000,0.0,0,portero,Real Madrid Club de Fútbol,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,P. Cuéllar,187000,0.0,0,portero,Real Club Deportivo Mallorca,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,A. López,164000,0.0,0,portero,Granada Club de Fútbol,14.0,813.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
496,I. Grbić,160000,0.0,0,portero,Club Atlético de Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,J. Doménech,160000,0.0,0,portero,Valencia Club de Fútbol,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,P. Reina,160000,0.0,0,portero,Villarreal Club de Fútbol,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
top_10_precio_Defensa

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points,Predicted_Price
63,A. Witsel,10736000,4.7,85,defensa,Club Atlético de Madrid,12.0,1010.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
76,J. Cancelo,16261000,5.0,80,defensa,Fútbol Club Barcelona,14.0,1258.0,0.0,1.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,NaN,NaN
79,S. Cardona,11393000,4.4,80,defensa,Unión Deportiva Las Palmas,14.0,705.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
93,M. Mármol,10103000,4.5,76,defensa,Unión Deportiva Las Palmas,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,D. Rico,10002000,4.9,68,defensa,Getafe Club de Fútbol,11.0,934.0,1.0,3.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
137,Ó. De Marcos,6181000,4.6,65,defensa,Athletic Club,14.0,1125.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN
176,G. González,1026000,3.3,56,defensa,Real Club Deportivo Mallorca,14.0,833.0,0.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
204,C. Azpilicueta,1627000,3.2,51,defensa,Club Atlético de Madrid,14.0,470.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
208,M. Diakhaby,1456000,4.2,50,defensa,Valencia Club de Fútbol,14.0,977.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN
218,R. Marín,2507000,3.3,49,defensa,Deportivo Alavés,13.0,546.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN


In [30]:
####